# Usage Guide

The following document is a guide to using the disk modeling code. It begins with a Quick Start guide, and progresses into more detail as to how to generate models.

## Quick Start Guide

Model images can be generated quickly using the default parameters in the modeling code:

In [3]:
import disk
x = disk.Disk() #creates a disk object using the default parameters

%run raytrace
total_model(x) #Runs radiative transfer using the disk object

The above code first creates a disk object using the default parameters.

`x = disk.Disk()`

This disk object contains information on the density, temperature, and velocity throughout the disk. In the next step the code runs radiative transfer on this disk object, again with the default parameters.

`total_model(x)`

The output will be a fits file called `testpy_alma.fits`. By default this is a 15 channel model, with 0.32 km/s channels, for a HD 163296-like disk (see Rosenfeld et al. 2013). 

## Beyond the default

The code is designed to create model images for a wide range of disk structures, observed by different molecules, in a wide variety of observational configurations. As such, there is built-in flexibility when creating the disk object, and when running the radiative transfer.

The creation of the disk object and the radiative transfer calculation are independent, such that any disk object that you create can be passed to the radiative transfer calculation without any additional specification. Similary, the same disk object can be imaged using different molecules and spectral/spatial resolutions. This flexibility is designed to allow for efficient modeling of multiple emission lines for a given disk, and for simplified exploration of the parameter space for a given disk.

In the space below we start by going through the creation of disk objects, followed by a discussion of variations in the radiative transfer calculation.

### Different disks

There are multiple different base codes that model disk objects (e.g., `disk.py`, `debris_disk.py`) that are used to model disks with different underlying parametric structures (see model.ipynb for more details).

Accessing these different base models can be done by importing the different pieces of code. Once the disk code has been imported, the syntax for creating a disk object is the same.

In [4]:
### Create a protoplanetary-like disk
import disk
ppd = disk.Disk()

### Create an eccentric protoplanetary-like disk
import disk_ecc
ppd_ecc = disk_ecc.Disk()

### Create a debris disk
import debris_disk
dd = debris_disk.Disk()

Each of the above disk objects (`ppd`, `ppd_ecc`, `dd`) can be sent to `total_model` to create model emission. 

Each of these disk objects has also been created assuming the default parameters for that model. While each model requires a slightly different set of input parameters, the method for specifying these parameters is the same for any type of disk object. This is done by specifying the value of the specific parameter that you want to set. For example, if you want to create a proto-planetary-like object with a temperature power law that goes as $r^{-0.15}$, and a gass mass of 0.01 M$_{sun}$, the syntax is:

In [5]:
ppd2 = disk.Disk(q=-0.15,McoG=0.01)

/Users/kevinflahertyastro/Documents/GitHub/disk_model3/disk.py:209: RuntimeWarning: divide by zero encountered in true_divide
  Omg = np.sqrt((dPdr/(rcf*self.rho0)+Disk.G*self.Mstar/(rcf**2+zcf**2)**1.5))
/Users/kevinflahertyastro/Documents/GitHub/disk_model3/disk.py:209: RuntimeWarning: invalid value encountered in true_divide
  Omg = np.sqrt((dPdr/(rcf*self.rho0)+Disk.G*self.Mstar/(rcf**2+zcf**2)**1.5))


In this case the temperature power law parameter and the gas mass have been changed, while the other parameters of the model have been kept at the default values. 

Similarly, we might want to change the eccentricity for the eccentric disk model, or the stellar luminosity used to calculate the gas and dust temperature in the debris disk model.

In [6]:
ppd_ecc2 = disk_ecc.Disk(ecc=0.05)

dd2 = debris_disk.Disk(Lstar=0.1) 

As mentioned above, each type of disk object has a slightly different set of input parameters. The list of input parameters, along with their default values (most often based on a HD 163296-like disk) are listed in the table below.

| Parameter | Description | disk.py | disk_pow.py | disk_ecc.py | debris_disk.py | 
|-----------|-------------|---------|-------------|-------------|----------------|
|`q`        | Temperature power law parameter | -0.5    | -0.5      | -0.5      | -0.5          |
|`McoG`     | Gas mass (in M$_{sun}$) | 0.09 | 0.09 | 0.09 | 0.09 |
|`pp`       | Surface density power law parameter | 1 | 1 | 1 | 1 |
|`Rc`       | Critical radius, in au[^1] | 150 | 150 | 150 | 150 |
|`incl`     | Inclination, in degrees | 51.5 | 51.5 | 51.5 | 51.5 |
|`Mstar`    | Stellar mass, in M$_{sun}$ | 2.3 | 2.3 | 2.3 | 2.3 |
|`Xco`      | CO abundance, relative to H$_2$ | 10$^{-4}$ | 10$^{-4}$ | 10$^{-4}$ | 10$^{-4}$ |
|`vturb`    | Turbulence, in units of thermal broadening | 0.01 | 0.01 | 0.01 | 0.01 |
|`Zq0`      | $T(z)$ parameter, in au | 33.9 | 33.9 | 33.9 | 33.9 |
|`Tmid0`    | Midplane temperature normalization, in K | 19 | 19 | 19 | N/A |
|`Tatm0`    | Atmosphere temperature normalization, in K | 69.3 | 69.3 | 69.3 | N/A |
|`sigbound` | Vertical boundaries for molecular abundance, as vertically integrated column density, in units of $\Sigma_{21}$ | [0.79,1000] | [0.79,1000] | [0.79, 1000] | [0.79,1000] |
|`Rabund`   | Inner and outer radii for molecular gas distribution, in au | [10,1000] | [10,1000] | [10,1000] | [10,1000] |
|`handed`   | Handedness of rotation, +1 or -1 | -1 | -1 | -1 | 
|`vtalpha`  | Radial power law index for turbulence | 0 | 0 | 0 | N/A |
|`vtsig`    | Vertical Gaussian width for turbulence | 0 | 0 | 0 | N/A |
|`Lstar`    | Stellar luminosity, in L$_{sun}$ | N/A | N/A | N/A | 1 |
|`sh_param` | Scale height parameter, =H/R | N/A | N/A | N/A | .1 |
|`ecc`      | Eccentricity of the disk | N/A | N/A | 0. | N/A |
|`aop`      | Angle of periapsis, in degrees | N/A | N/A | 0. | N/A |

|Parameter | Description |  disk.py | disk_pow.py | disk_ecc.py | debris_disk.py | 
|----------|-------------|---------|-------------|-------------|----------------|
|`nr`      | Number of grid cells in the radial direction in the radiative transfer grid | 180 | 180 | 180 | 180 |
|`nphi`    | Number of grid cells in the azimuthal direction in the radiative transfer grid | 131 | 131 | 131 | 131 |
|`nz`      | Number of grid cells in the vertical direction in the radiative transfer grid | 300 | 300 | 300 | 300 |
|`zmax`    | Maximum vertical extent of the computational domain, in au | 170 | 170 | 170 | 170 |
|`Rin`      | Inner radial edge of computational domain, in au | 1 | 10 | N/A | 10 |
|`Rout`     | Outer radial edge of computational domain, in au | 1000 | 1000 | N/A | 1000 |
|`Ain`      | Inner radial edge of the computational domain, in au | N/A | N/A | 10 | N/A |
|`Aout`     | Outer radial edge of the computational domain, in au | N/A | N/A | 1000 | N/A |
|`r_gridsize`| Number of radial grid cells when calculating density, temperature | N/A | N/A | N/A | 500 |
|`z_gridsize`| Number of vertical grid cells when calculating density, temperature | N/A | N/A | N/A | 500 |


In addition to input parameters with numerical values, there are other parametes that control e.g. how turbulence is treated 

|Parameter | Description |  disk.py | disk_pow.py | disk_ecc.py | debris_disk.py | 
|----------|-------------|---------|-------------|-------------|----------------|
|`rtg`     |Calculate the radiative tranfer grid when creating the disk object? | True | True | True | True|
|`vcs`  |Treat turbulence as protportional to the sound speed?[^2] | True | True | True | True|
|'`sh_relation` | How H is related to R[^3] | N/A | N/A | N/A | `'linear'` |
|`line` | Molecular line to model[^4] | `'co'` | `'co'` | `'co'` | `'co'` |
|`ring` | Add a ring?[^5] | `None` | `None` | `None` | `None` |
|`annulus` | Include an annulus[^6] | N/A | N/A | N/A | `None` |
|`exp_temp` | Use an exponential vertical temperature profile | False | False | N/A | N/A |
|`include_selfgrav` | Include disk self-gravity in the orbital velocity calculation | False | False | N/A | N/A |



[^1]: For `disk_pow.py`, `Rc` specifies the outer edge of the disk. 
[^2]: If False, the `vturb` value is treated as the turbulence in units of km/s.
[^3]: Possible values are `'linear'` or `'const'`.
[^4]: Possible values are `'co'`, `'c18p'`, `'13co'`, `'hco'`, `'hcn'`, `'cs'`, `'dco'`.
[^5]: Set this parameter to a list of [Rring,Wring,Sig_Enhance] to include a corresponding additional ring of emission, where Rring is the center radius of the ring (in au), Wring is the width of the ring (in au), and Sig_Enhance is ???
[^6]: Set this parameter to [Rin,Rout,mass] to include an additional annulus of material, with an inner radius Rin (in au), an outer radius of Rout (in au) and a total mass of mass (in Earth masses).

### Varying Model Image

Once a disk object is created, the function `total_model` can create a set of channel maps. The exact apperance of the disk depends on the number of channels, the exact line used, the size of the image, etc., all of which are specified within `total_model`. These parameters can be broken down into those related to the spectrum, the image, the line, and extra parameters.


**Spectral Parameters**

|Parameter | Description | Default value | 
|----------|-------------|---------|
|`chanmin` | Minimum velocity, in km/s | -2.24 |
|`nchans`  | Number of channels | 15 |
|`chanstep`| Channel width, in km/s | 0.32 |
|`vsys` | Systemic velocity, in km/s | 5.79 |
|`obsv`[^1] | List of velocities for the channels in the data, in km/s | None |
|`flipme` | Mirror the spectrum about the central channel?[^2] | True |
|`bin` | The factor by which to bin down the spectrum | 1 |
|`hanning` | Apply Hanning smoothing in the spectral direction | True |
|`l_cloud` | Correct for absorption by an intervening cloud? | False |
|`tau` | List of maximum optical depths of intervening clouds | `[0,]` |
|`sigma_c` | List of Gaussian cloud widths, in km/s | `[6,]` |
|`velocity_c` | List of central velocities for the intervening clouds, in km/s | `[2,]` |

**Image Parameters**
|Parameter | Description | Default Value |
|----------|-------------|---------------|
|`imres` | Width of a pixel, in arc-seconds | 0.05 |
|`distance`| Distance to the object, in parsecs | 122 |
|`PA` | Position angle, east of north, in degrees | 312.46 |
|`offs` | List of RA and Dec positional offsets for the disk, in arc-seconds | `[0.0,0.0]` |
|`xnpix` | Number of pixels along the x-direction of the image (the image is assumed to be square) | 512 |

**Line Parameters**
|Parameter | Description | Default Value | 
|----------|-------------|---------------|
|`Jnum`   | Lower J quantum number of the transition | 2 |
|`freq0` | Rest frequency of the transition | 345.79599 |
|`abund` | Relative abundance of this molecule | 1 | 

**Other parameters**
|Parameter | Description | Default Value |
|----------|-------------|---------------|
|`wind`  | Include a wind? | False |
|`isgas` | Model gas emission? | True |
|`includeDust` | Include dust emission? | False |
|`extra` | Perform extra calculations?[^3] | 0 |
|`modfile` |Base of the model file name | `'testpy_alma'`|


[^1]: If specified, the model images will be interpolated onto the velocities in obsv, after adding the systemic velocity. 
[^2]: If True, this assumes that the central channel corresponds to zero velocity. Otherwise the reflection will be invalid.
[^3]: Valid values are 1 (plot a figure showing the disk structure with the tau=1 surface marked with a dashed line), 2.1 (print a list of heights that represent the Nth percentile of where the emission comes from at that radius), 2.2 (prints a list of temperatures that represent the Nth percentile of the gas temperature where the emission comes from at that radius), 3.0 (create channel maps where the pixel value is the height, in au, of the tau=1 surface), 3.1 (create channel maps where the pixel value is the temperature, in K, of the tau=1 surface), 3.2 (create channel maps where the pixel values are the maximum optical depth)

Other things to describe in more detail:
- the ring parameter in disk.py
- the sigbound parameter in disk.py
- how the velocities work, especially with flipme and obsv, in total_model
- the extra parameter
- hanning smoothing and binning
- isgas and includeDust, and when to use which one.
- the vertical temperature profile for exp_temp
- List the valid transitions that it uses
- the functions within disk.py and what they do